# Creating a Guitar Recommendation Engine using clustering - Part 1

#### Reece Vincent
#### 05/04/2023


## Contents
#### 0 - Data Gathering
- **0.1 Webscraping process**
- **0.2 Webscraping Script**

#### 1 - Data Consolidation
- **1.1 Introduction**
- **1.2 Loading and examining data**
- **1.3 Data consolidation**

#### 2 - EDA and FE
- **2.1. Data Preparation**
- **2.2 EDA and feature Analysis**
- **2.3 Test Model**

#### 3 - Conclusions

## 1.1 Introduction

The electric guitar market is vast and can be overwhelming for novice guitarists or even experienced players looking to expand their collections. To alleviate this problem, I will create a basic Recommendation Engine for Electric Guitars, using data webscraped from https://www.guitarguitar.co.uk/ (for more info see "0 - Data Gathering" section). The recommendation engine will be built on machine learning techniques, specifically unsupervised clustering, to suggest the most similar guitar to the user's inputted model.

In the first stage of the project, I will examine the scraped data and highlight issues with the format, which I will then partially resolve in the Data Consolidation section. I will then explore the data and select relevant features for the modeling section.

The modeling stage will utilize unsupervised clustering to group similar guitars together into classes. This clustering will form the basis of the recommendation engine, where the most similar guitar within a user's selected class will be suggested. How similarity is defined is discussed in the modelling section. Additionally, I have also included an output of the most dissimilar guitar in the same class as the inputted model to provide some variation.

Finally, I will discuss the limitations of this project, including the scope of the dataset, the performance of the clustering model, and potential avenues of exploration and improvement for future projects.

In [53]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re #RegEx for pattern matching

## 1.2 Loading and examining data


In [2]:
## Import csv and head check
df = pd.read_csv('guitars.csv')
df.head(10)

C:\Users\gabri\AppData\Local\Temp\ipykernel_9296\4211228351.py:2: DtypeWarning: Columns (7,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,146,147,148,149,150,153,154,158,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,181,182,183,184,185,186,187,188,189,190,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,28

,brand,model,link,rating,id,Body,Neck,Profile,Nut Width,Fingerboard,...,Bass Pickup,"2 Volumes, 2 Push/Pull Tone Controls, 3-way Toggle Switch",PRS Two-Piece Stoptail Hybrid Hardware,Phase III Vintage Tuners with Faux Bone Buttons,Top Material\t,Back & Sides\t,Neck Material\t,Saddle\t,Electronics\t,Tuners
0,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRS,PRS SE John Mayer Silver Sky Dragon Fruit,https://www.guitarguitar.co.uk/product/2112143...,NaN,Manufacturer's ID: SEJ2R1J,Poplar,Maple,NaN,1 20/32”,Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Epiphone,Epiphone Les Paul Standard '60s Iced Tea,https://www.guitarguitar.co.uk/product/1912163...,4.8/5,Manufacturer's ID: EILS6ITNH1,Mahogany with AA Flame Maple Top,Mahogany,NaN,"1.693""",Indian Laurel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fender,Fender Kurt Cobain Jaguar 3 Colour Sunburst NO...,https://www.guitarguitar.co.uk/product/1312241...,5/5,Manufacturer's ID: 0143001700,Alder,Maple,NaN,"1.65""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Epiphone,Epiphone SG Standard Ebony,https://www.guitarguitar.co.uk/product/1912163...,5/5,Manufacturer's ID: EISSBEBNH1,Mahogany,Mahogany,NaN,"1.693""",Indian Laurel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Pensa Guitars,Pensa Guitars 10th Anniversary MK90 #1034,https://www.guitarguitar.co.uk/product/2210213...,NaN,NaN,Chambered Swamp Ash,Hand Selected Birdseye Maple,NaN,"1-5/8""",East Indian rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Epiphone,Epiphone Les Paul Standard '50s Heritage Cherr...,https://www.guitarguitar.co.uk/product/1912163...,5/5,Manufacturer's ID: EILS5HSNH1,Mahogany with Maple Cap,Mahogany,NaN,"1.693""",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Rickenbacker,Rickenbacker 330 Jetglo Left Handed,https://www.guitarguitar.co.uk/product/1211121...,NaN,Manufacturer's ID: 50330L-JG,Maple,Maple,NaN,"1.63""",Chechen (Caribbean Rosewood),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Fender,Fender Player Stratocaster Polar White Maple F...,https://www.guitarguitar.co.uk/product/1804303...,4.8/5,Manufacturer's ID: 0144502515,Alder,Maple,NaN,"1.650""",Maple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Epiphone,Epiphone Les Paul Custom Ebony,https://www.guitarguitar.co.uk/product/1912163...,4.9/5,Manufacturer's ID: EILCEBGH1,Mahogany,Mahogany,NaN,"1.693""",Ebony,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## Large amount of columns
df.shape

(3211, 571)

In [4]:
## Null check
df.isnull().sum()

brand                 0
model                 0
link                  0
rating             2365
id                  329
                   ... 
Back & Sides\t     3209
Neck Material\t    3209
Saddle\t           3209
Electronics\t      3209
Tuners             3209
Length: 571, dtype: int64

In [5]:
## df too large to return info properly
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3211 entries, 0 to 3210
Columns: 571 entries, brand to Tuners        
dtypes: float64(20), object(551)
memory usage: 14.0+ MB


Immediately it is apparent that there is an issue with the Dataframe's dimensions. There is a significant amount of columns and a large amount of these are mostly null. This will be looked into further but first a quick check of duplicates takes place.

In [6]:
## Only 1 pure duplicate
df[df.duplicated(keep=False)]

,brand,model,link,rating,id,Body,Neck,Profile,Nut Width,Fingerboard,...,Bass Pickup,"2 Volumes, 2 Push/Pull Tone Controls, 3-way Toggle Switch",PRS Two-Piece Stoptail Hybrid Hardware,Phase III Vintage Tuners with Faux Bone Buttons,Top Material\t,Back & Sides\t,Neck Material\t,Saddle\t,Electronics\t,Tuners
992,Schecter,Schecter PT Special Purple Burst Pearl,https://www.guitarguitar.co.uk/product/2101203...,NaN,Manufacturer's ID: SCH-GTR-667,Swamp Ash,Maple,NaN,"1.653""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,Schecter,Schecter PT Special Purple Burst Pearl,https://www.guitarguitar.co.uk/product/2101203...,NaN,Manufacturer's ID: SCH-GTR-667,Swamp Ash,Maple,NaN,"1.653""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
## Drop the duplicate
df = df.drop_duplicates().reset_index(drop=True)
df.shape

(3210, 571)

In [8]:
## Get value counts for the 'model' column
value_counts = df['model'].value_counts()

## Filter value counts to only include values > 1
value_counts_greater_than_1 = value_counts[value_counts > 1]


## Get amount of duplicates and list of them - 15 duplicate pairs (30 entries)
print(f'There are {len(value_counts_greater_than_1)} duplicate model names')
print()
print(value_counts_greater_than_1)

There are 15 duplicate model names

Solar Guitars A2.6FRPN Canibalismo                               2
Solar Guitars S1.6AQOB Trans Ocean Blue                          2
Schecter KM-6 MK-III Keith Merrow Trans Black Burst              2
Schecter Solo-II Standard Faded Vintage Sunburst                 2
Schecter Sun Valley SS FR III Sky Blue Burst                     2
Schecter Omen Extreme 6 Vintage Sunburst                         2
Schecter PT Pro Ebony Trans Purple Burst                         2
Schecter Omen Elite-6 See-Thru Blue Burst                        2
Schecter PT Special Aqua Burst Pearl                             2
Schecter Nick Johnston Traditional HSS Atomic Ink                2
Solar Guitars A1.6C Carbon Matte Black                           2
Schecter Synyster Gates Standard Black with Silver Pinstripes    2
Schecter SOLO-II FR Apocalypse Red Reign                         2
Steinberger Spirit GT-Pro Deluxe (HB-SC-HB) White                2
Epiphone Tom DeLonge ES-33

I wish to use the model names as unique identifiers however there are 15 duplicate Guitar Model names. Please note some further investigation may yield that these are in fact separate models with different features and some renaming could be carried out to preserve unique entries and ensure unique names. However, that is not done during this project and they are instead dropped completely - such a small sample being dropped will have a negligible impact on the data. This ensures each model entry is now a unique identifier.

In [9]:
## Drop model duplicates
df = df.drop_duplicates(subset=['model'], keep='first').reset_index(drop=True)

In [11]:
## ID also has duplicates
df.id.value_counts()

Manufacturer's ID: LPS500HSNH1    35
Manufacturer's ID: LPS500TONH1    23
Manufacturer's ID: LPS600B8NH1    16
Manufacturer's ID: LPS600ITNH1    13
Manufacturer's ID: LPS600UBNH1    10
                                  ..
Manufacturer's ID: 0149872372      1
Manufacturer's ID: 110232849       1
Manufacturer's ID: 110232807       1
Manufacturer's ID: 110332829       1
Manufacturer's ID: 110392800       1
Name: id, Length: 2431, dtype: int64

In [12]:
## Inspect some entries

df[df['id'] == 'Manufacturer\'s ID: LPS500HSNH1']

,brand,model,link,rating,id,Body,Neck,Profile,Nut Width,Fingerboard,...,Bass Pickup,"2 Volumes, 2 Push/Pull Tone Controls, 3-way Toggle Switch",PRS Two-Piece Stoptail Hybrid Hardware,Phase III Vintage Tuners with Faux Bone Buttons,Top Material\t,Back & Sides\t,Neck Material\t,Saddle\t,Electronics\t,Tuners
0,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990,Gibson,Gibson Les Paul Standard 50s Heritage Cherry S...,https://www.guitarguitar.co.uk/product/1903193...,4.9/5,Manufacturer's ID: LPS500HSNH1,"Mahogany, Maple (Top)",Mahogany,Vintage '50s,"1.69""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looking at the above we see some of these models are actually the same but only differ in variations of the name. There are other examples of this whereby the name may contain some additional cosmetic info (e.g a colour the guitar is) but apart from that it is the exact same as another model.

As cosmetic properties are being treated as secondary features during this project and as I want entries to be as unique as possible I will remove duplicates of any models which have the same Manufacturer's ID and only differ by model name and link. Like the model name duplicates from earlier this is something that some more extensive investigation could be done on to avoid data loss. I believe the approach I have opted for is sufficient for this project.

In [13]:
### Get value counts for the 'id' column
value_counts = df['id'].value_counts()

## Filter value counts to only include values > 1
value_counts_greater_than_1 = value_counts[value_counts > 1]

## See how many there are
len(value_counts_greater_than_1)

240

In [14]:
## Create a loop that finds and drops the duplicates
## Initialise a counter
k = 0

## Iterate the model names for entries with Manufacturer's ID value count >1
for i in value_counts_greater_than_1.index:

    ## Get all rows for id "i" , drop model and link columns and mask duplicates
    df_model = df[df['id'] == i].drop(['model', 'link'], axis=1)
    duplicates = df_model.duplicated()


    ## Drop duplicated rows from main df
    if duplicates.any():
        df.drop(df_model[duplicates].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        k+=1
    else:
        pass

## See how many of the 240 duplicate ids had genuine duplicates
print(f'Check complete: {k} id entries had pure duplicates')

Check complete: 62 id entries had pure duplicates


In [15]:
## 196 entries were dropped
df.shape

(3014, 571)

In [16]:
## One price null (can obtain this data manually using the weblink)
df[df['price'].isnull() == True]

,brand,model,link,rating,id,Body,Neck,Profile,Nut Width,Fingerboard,...,Bass Pickup,"2 Volumes, 2 Push/Pull Tone Controls, 3-way Toggle Switch",PRS Two-Piece Stoptail Hybrid Hardware,Phase III Vintage Tuners with Faux Bone Buttons,Top Material\t,Back & Sides\t,Neck Material\t,Saddle\t,Electronics\t,Tuners
3013,Epiphone,Epiphone 2011 Epiphone Tom DeLonge ES-333 Limi...,https://www.guitarguitar.co.uk/product/pxp2301...,NaN,NaN,Laminated Maple,Mahogany,NaN,"1.68""",Rosewood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df.loc[3013, 'price'] = '£989.00'

In [18]:
## Can see some features have made their way in - due to errors on product pages
print(df.columns)

Index(['brand', 'model', 'link', 'rating', 'id', 'Body', 'Neck', 'Profile',
       'Nut Width', 'Fingerboard',
       ...
       'Bass Pickup',
       '2 Volumes, 2 Push/Pull Tone Controls, 3-way Toggle Switch',
       'PRS Two-Piece Stoptail Hybrid Hardware',
       'Phase III Vintage Tuners with Faux Bone Buttons', 'Top Material\t',
       'Back & Sides\t', 'Neck Material\t', 'Saddle\t', 'Electronics\t',
       'Tuners        '],
      dtype='object', length=571)


Examining the large number of columns it is clear that there are two main reasons for this. Firstly, some specs represent the same quality but have different names or inconsistencies in their formatting. Examples are shown in a moment.

Secondly, some specification values have been mapped as columns. This was not due to an error with webscraping but because of inconsistencies on the formatting of the https://www.guitarguitar.co.uk/ website. As over 3000 entries were scraped it would be extremely time-consuming (and likely inefficient) to attempt to go back and make a large amendments to the webscraping script. By this I mean that each page would need to be manually examined and changes made in the script for each product page that has problems (and this could be a large amount).

Instead, Data Consolidation will be needed. While this might less time-consuming there is a trade-off that some entries may have to be omitted.

## 3. Data consolidation

As mentioned, some columns in the dataset are essentially duplicates representing the same features but being called something different for different models. There are possibilities for this dataframe to be consolidated. I'll now examine how this may be done and as before this could be carried out more extensively to retain complete some features that will not be included in the final dataframe. To highlight the duplication of features lets look at the different type of frets columns that are present:

In [19]:
## Find column names containing variations of 'fret' or ending with punctuation
## There seems to be a mixture of features these columns refer to such as number of frets on the guitar or fretboard material.
## This also captures columns that are referring to guitar measurements at certain frets

fret_columns = df.columns[df.columns.str.contains(r'fret\w*[^\w\s]?', regex=True, case=False)]
print(fret_columns)

Index(['Frets', 'Fretboard', 'Fretboard Radius', 'Fretboard Material',
       ' Fretboard', ' Fret', 'No of Frets', 'Number of Frets', 'Frets Size',
       'Fret', 'Width at Last Fret', 'Frets:', 'fretboard', 'fret',
       'Fret Size', 'number of frets', 'Fretboard Radius (mm)',
       'Number of Frets ', 'Fret Size ', 'FRETWIRE', 'FRETS', 'Frets: ',
       'Fretboard Radius  ', 'Thickness @ 1st Fret', 'Fretboard\t', 'Frets\t',
       'Frets/Type', 'Fretwire', 'Neck Thickness at 1st Fret',
       'Neck Thickness at 12th Fret', 'Fret Wire',
       'Neck Width @ Nut/12th Fret', 'Thickness @1st/12th Fret',
       '12th Fret Width', 'Number Of Frets', 'No. Frets',
       'Neck Width at 12th Fret', 'Fret Marker Style', 'Number of Frets:',
       'Fret Size:', ' Fretboard  ', ' Fret  ', '\nFretboard Material',
       '\nFrets', '\nFretboard Radius', 'Fret Markers', 'Fretboard Wood'],
      dtype='object')


In [20]:
## Examine value counts of some of these columns
df['Frets'].value_counts()

22                   530
22 Medium Jumbo      340
24                   169
21                   158
24 Jumbo             135
                    ... 
22 XJ                  1
 22, Medium Jumbo      1
Super Jumbo            1
22 Medium              1
21 Medium Vintage      1
Name: Frets, Length: 238, dtype: int64

In [21]:
df['Number of Frets '].value_counts()

21.0    1
Name: Number of Frets , dtype: int64

In [22]:
df[df['Number of Frets '] == 21.0][['Number of Frets ', 'Frets']]

,Number of Frets,Frets
567,21.0,NaN


As shown above the columns 'Number of Frets' and 'Frets' are representing the same feature. In such cases, consolidation can be highly advantageous. However, it is important to note that not all examples will be as straightforward as this one. It is crucial to ensure that consolidation is not done in cases where clashes occur. For instance, if a row has entries in both columns, they should not be consolidated into a single column without further inspection. Therefore, it is essential to conduct a thorough analysis before consolidation takes place. First I show some high-level similarities in column names:

In [23]:
## Check for exact duplicates of column names
duplicated_cols = df.columns[df.columns.duplicated(keep=False)]

if len(duplicated_cols) > 0:
    print(f'The following columns are duplicated: {duplicated_cols}')
else:
    print('No duplicates')

No duplicates


In [24]:
## Same again but we lowercase columns beforehand
duplicated_low = df.columns[df.columns.str.lower().duplicated(keep=False)]

if len(duplicated_low) > 0:
    print(f'The following columns are duplicated: {duplicated_low}')
else:
    print('No duplicated columns found')

## print no. of duplicates
len(duplicated_low)

The following columns are duplicated: Index(['model', 'Body', 'Neck', 'Nut Width', 'Radius', 'Scale', 'Frets',
       'Bridge', 'Tuners', 'Pickups', 'Fretboard', 'Inlays', 'Binding',
       'Bridge Pickup', 'Neck Pickup', 'Tailpiece', 'Hardware',
       'Country of Origin', 'Hardware Color', 'Neck type', 'Neck pickup',
       'Bridge pickup', 'Factory tuning', 'String gauge', 'Hardware color',
       'Pickguard', 'Strap Buttons', 'Color', 'Number of Frets',
       'Middle Pickup', 'Control Knobs', 'Strings', 'Neck Type',
       'Control knobs', 'Fret', 'neck type', 'body', 'fretboard', 'fret',
       'bridge', 'neck pickup', 'bridge pickup', 'hardware color',
       'number of frets', 'middle pickup', 'factory tuning', 'string gauge',
       'tuners', 'Country Of Origin', 'Middle pickup', 'Electronics',
       'NECK PROFILE', 'SHAPE', 'BINDING', 'STRINGS', 'FRETWIRE', 'SCALE',
       'FRETS', 'RADIUS', 'SIDE DOTS', 'PICKGUARD', 'PICKUP NECK',
       'PICKUP BRIDGE', 'colour', 'neck pu'

95

In [25]:
## Now extend this by removing l and r whitespace (this also captures esc characters) - 89 more
duplicated_strip = df.columns[df.columns.str.strip().duplicated(keep=False)]

if len(duplicated_strip) > 0:
    print(f'The following columns are duplicated: {duplicated_strip}')
else:
    print('No duplicated columns found')

len(duplicated_strip)

The following columns are duplicated: Index(['Body', 'Neck', 'Nut Width', 'Fingerboard', 'Radius', 'Scale', 'Frets',
       'Nut', 'Bridge', 'Tuners',
       ...
       '\nHardware Color', '\nBridge', '\nTuners', '\nKnobs', '\nStrings',
       'Top Material\t', 'Back & Sides\t', 'Neck Material\t', 'Electronics\t',
       'Tuners        '],
      dtype='object', length=184)


184

To consolidate columns that only differ in name by casing, whitespace, punctuation and/or escape characters I create a consolidation dictionary. The idea behind this is that the root word is identified by applying the string transformations just shown and those are used as keys. The original names that transform into the roots are stored as the values for their respective keys.

In [26]:
## Create empty duplicate dictionary
duplicates_dict = {}

## Iterate through column names
for col in df.columns:
    col_clean = col.lower()  ## Lowercases transformation
    col_clean = col_clean.strip()  # Strips leading and trailing whitespace
    col_clean = col_clean.rstrip(':')  # Strips any trailing colons

    ## Column name and Transformed column name become k-v pair if k doesn't already exist
    if col_clean not in duplicates_dict:
        duplicates_dict[col_clean] = [col]
    ## If respective k exists untransformed col name is appended as value
    else:
        if col not in duplicates_dict[col_clean]:
            duplicates_dict[col_clean].append(col)

## Only want duplicates
duplicates = {k: v for k, v in duplicates_dict.items() if len(v) > 1}

## View what has been captured
if len(duplicates) > 0:
    print("The following columns are duplicated:")
    for k, v in duplicates.items():
        print(f"{v} are duplicates of {k}")
else:
    print("No duplicated columns found.")



The following columns are duplicated:
['model', 'Model'] are duplicates of model
['Body', ' Body', 'Body ', 'body', 'Body:\xa0', '\tBody\t', 'Body\t', 'BODY', ' Body  '] are duplicates of body
['Neck', 'Neck ', '\tNeck\t', 'Neck\t', 'NECK'] are duplicates of neck
['Nut Width', 'Nut Width:', 'Nut Width ', '\tNut Width\t', 'Nut Width\t', 'Nut width', '\nNut Width'] are duplicates of nut width
['Fingerboard', 'Fingerboard ', 'Fingerboard:\xa0', '\tFingerboard\t', 'Fingerboard:'] are duplicates of fingerboard
['Radius', 'RADIUS', '\tRadius\t', 'Radius\t'] are duplicates of radius
['Scale', 'Scale ', 'SCALE', '\nScale'] are duplicates of scale
['Frets', 'Frets:', 'FRETS', 'Frets:\xa0', 'Frets\t', '\nFrets'] are duplicates of frets
['Nut', 'Nut ', 'Nut:', '\nNut'] are duplicates of nut
['Bridge', ' Bridge', 'Bridge:', 'bridge', 'Bridge:\xa0', '\tBridge\t', 'Bridge\t', 'BRIDGE', ' Bridge  ', '\nBridge'] are duplicates of bridge
['Tuners', 'tuners', '\tTuners\t', 'Tuners\t', 'TUNERS', '\nTuner

In [27]:
## View how many cols were consolidated
## Get the number of keys in the dictionary
num_keys = len(duplicates)
print(f'Number of keys: {num_keys}')

# Get the number of values in the dictionary
num_values = 0
for key in duplicates:
    num_values += len(duplicates[key]) if isinstance(duplicates[key], list) else 0

## Print consolidation values
print(f'Number of values: {num_values}')
print()
print(f'{num_values} columns could consolidated to {num_keys} columns which is a reduction of {num_values - num_keys} columns.')

Number of keys: 100
Number of values: 318

318 columns could consolidated to 100 columns which is a reduction of 218 columns.


Before performing this consolidation I check for clashes using masking as mentioned earlier. For clarity this is demonstrated on one column type (namely 'bridge') and will then be used more generally. In this example ['Bridge', 'bridge', 'BRIDGE'] are values in the duplicates dictionary corresponding to the 'bridge' key.

In [29]:
## Demonstrate on mini df
df_mini = df[['Bridge', 'bridge', 'BRIDGE']]

## Use .notna() to return boolean depending on if an entry has a value - add col with row sum
df_mini['sum_of_Bridges'] = df_mini[['Bridge', 'bridge', 'BRIDGE']].notna().sum(axis=1)

C:\Users\gabri\AppData\Local\Temp\ipykernel_9296\2436885037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mini['sum_of_Bridges'] = df_mini[['Bridge', 'bridge', 'BRIDGE']].notna().sum(axis=1)


In [30]:
df_mini.head()

,Bridge,bridge,BRIDGE,sum_of_Bridges
0,ABR-1 Tune-O-Matic w/ Stopbar,NaN,NaN,1
1,2-Point Steel Tremolo,NaN,NaN,1
2,LockTone ABR with Stopbar,NaN,NaN,1
3,Black Chrome Adjusto-Matic w/ Chrome Vintage-S...,NaN,NaN,1
4,LockTone ABR Tune-o-Matic with Stopbar,NaN,NaN,1


In [31]:
## 139 bridge entries are still outstanding - could be more columns that need consolidating or genuine nulls
df_mini['sum_of_Bridges'].value_counts()

1    2875
0     139
Name: sum_of_Bridges, dtype: int64

In [32]:
## Check sum_of_bridges value counts
df_mini[df_mini['sum_of_Bridges'] < 1].count()

Bridge              0
bridge              0
BRIDGE              0
sum_of_Bridges    139
dtype: int64

In [33]:
## vast majortity of bridge values came from "Bridge"
df_mini[df_mini['sum_of_Bridges'] == 1].count()

Bridge            2864
bridge              10
BRIDGE               1
sum_of_Bridges    2875
dtype: int64

In [35]:
## The key finding is that there are no clashes in consolidated values
df_mini[df_mini['sum_of_Bridges'] > 1].count()

Bridge            0
bridge            0
BRIDGE            0
sum_of_Bridges    0
dtype: int64

With this concept demonstrated I will now proceed to apply the Boolean sum consolidation generally across all k-v pairs in the duplicate dictionary for each row in the dataframe. This will be returned as a new dataframe consisting of sum columns for all the consolidations with each row representing each row in the original dataframe. This dataframe will then be checked for any clashes.

In [36]:
## Create a new DataFrame with "sum_of_k" columns for each key in duplicates
sum_df = pd.DataFrame(columns=['sum_of_'+k for k in duplicates.keys()])

## Iterate over the rows in df
for i, row in df.iterrows():
    ## Initialize a new row dictionary with zeros for all "sum_of_k" columns
    new_row = {col: 0 for col in sum_df.columns}
    ## Iterate over the items in the duplicates dictionary
    for k, v in duplicates.items():
        ## Count the number of non-null values in the selected columns for the current row
        count = row[list(v)].count()
        ## Add the count to the corresponding "sum_of_k" column for the current row
        new_row['sum_of_'+k] += count

    ## Add the new row to test_df with the appropriate index value
    sum_df.loc[i] = new_row

## Head check
sum_df.head()

,sum_of_model,sum_of_body,sum_of_neck,sum_of_nut width,sum_of_fingerboard,sum_of_radius,sum_of_scale,sum_of_frets,sum_of_nut,sum_of_bridge,...,sum_of_body binding,sum_of_finish package,sum_of_plating,sum_of_scratch plate,sum_of_neck construction,sum_of_strap button,sum_of_body top wood,sum_of_neck binding,sum_of_bracing,sum_of_body back
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [38]:
## Create boolean mask where any "sum_of_k" column greater than 1 is True
mask = (sum_df.filter(like='sum_of_') > 1).any(axis=1)

## Filter test_df to only show those rows
filtered_df = sum_df[mask]

## Print the filtered DataFrame
filtered_df

,sum_of_model,sum_of_body,sum_of_neck,sum_of_nut width,sum_of_fingerboard,sum_of_radius,sum_of_scale,sum_of_frets,sum_of_nut,sum_of_bridge,...,sum_of_body binding,sum_of_finish package,sum_of_plating,sum_of_scratch plate,sum_of_neck construction,sum_of_strap button,sum_of_body top wood,sum_of_neck binding,sum_of_bracing,sum_of_body back
2292,2,0,0,0,0,0,0,1,1,1,...,1,0,0,0,0,1,0,0,0,0


There is only one clash in the whole df. Examining it shows that we have two 'model' entries that differ slightly as one is more detailed but the extra detail is nonessential. This can easily be rectified later and is a good sign the consolidation checks are working as intended.

In [40]:
## We see that we have two model variables that differ slightly as one is more detailed but nonessential.
## This can easily be rectified later and is a good sign the consolidation checks are working as intended.

df.iloc[2292]

brand                                                           Suhr
model              Suhr guitarguitar Select #160 Standard Carve T...
link               https://www.guitarguitar.co.uk/product/1910113...
rating                                                           NaN
id                                                               NaN
                                         ...                        
Back & Sides\t                                                   NaN
Neck Material\t                                                  NaN
Saddle\t                                                         NaN
Electronics\t                                                    NaN
Tuners                                                           NaN
Name: 2292, Length: 571, dtype: object

In [42]:
## Set to null
df.loc[2292, 'Model'] = np.nan

### Consolidated dataframe

The consolidated dataframe can now be created. It is important to note that there may be further opportunities for consolidation that could allow more features to be used in the modeling process while minimising data loss. Due to project timeline constraints further such consolidation was not achieved here.

In [44]:
## Create consolidated df columns from duplicate keys
## As not all cols were consolidated any other cols need to be added manually
## As'price' and 'brand' are likely useful features these are added

cons_df = pd.DataFrame(columns=[k for k in duplicates.keys()] + ['price', 'brand'])

## Iterate over the rows in df
for i, row in df.iterrows():
    ## Initialize a new row dictionary with NaN for all "k_cons" columns
    new_row = {col: np.nan for col in cons_df.columns}
    ## Add the Price, ID and Brand for row values to the new row dictionary
    new_row['price'] = row['price']
    new_row['brand'] = row['brand']
    new_row['id'] = row['id']
    ## Iterate over the items in the duplicates dictionary
    for k, v in duplicates.items():
        ## Get the non-null value from the selected columns for the current row
        non_null_values = row[list(v)].dropna()
        if len(non_null_values) == 1:
            ## Only one non-null value found, store it in the corresponding "k_cons" column for the current row
            new_row[k] = non_null_values.iloc[0]
        ## Further consistency check to catch any errors
        elif len(non_null_values) > 1:
            print(f'Error on {i}')
    ## Add the new row to cons_df with the appropriate index value
    cons_df.loc[i] = new_row

## Head check
cons_df.head()

,model,body,neck,nut width,fingerboard,radius,scale,frets,nut,bridge,...,plating,scratch plate,neck construction,strap button,body top wood,neck binding,bracing,body back,price,brand
0,Gibson Les Paul Standard 50s Heritage Cherry S...,"Mahogany, Maple (Top)",Mahogany,"1.69""",Rosewood,"12""","24.75""","22, Medium Jumbo",Graph Tech,ABR-1 Tune-O-Matic w/ Stopbar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,£599.00,Gibson
1,PRS SE John Mayer Silver Sky Dragon Fruit,Poplar,Maple,1 20/32”,Rosewood,8.5”,25.5”,22,Synthetic Bone,2-Point Steel Tremolo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,£469.00,PRS
2,Epiphone Les Paul Standard '60s Iced Tea,Mahogany with AA Flame Maple Top,Mahogany,"1.693""",Indian Laurel,"12""","24.75""",22 Medium Jumbo,Graph Tech,LockTone ABR with Stopbar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"£1,399.00",Epiphone
3,Fender Kurt Cobain Jaguar 3 Colour Sunburst NO...,Alder,Maple,"1.65""",Rosewood,"9.5""","24""","22, Medium Jumbo",Synthetic Bone,Black Chrome Adjusto-Matic w/ Chrome Vintage-S...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,£399.00,Fender
4,Epiphone SG Standard Ebony,Mahogany,Mahogany,"1.693""",Indian Laurel,"12""","24.75""",22 Medium Jumbo,Graph Tech,LockTone ABR Tune-o-Matic with Stopbar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"£9,499.00",Epiphone


In [45]:
cons_df.isnull().sum()

model              0
body             217
neck             284
nut width        759
fingerboard      325
                ... 
neck binding    3010
bracing         3011
body back       3011
price              0
brand              0
Length: 102, dtype: int64

In [46]:
## Consolidation has reduced nulls in some cases if just one column had been chosen e.g
## Body 249 -> body 217
## Frets 209 -> frets 115
## Bridge 150 -> bridge 40

df.isnull().sum()

brand                 0
model                 0
link                  0
rating             2260
id                  328
                   ... 
Back & Sides\t     3012
Neck Material\t    3012
Saddle\t           3012
Electronics\t      3012
Tuners             3012
Length: 571, dtype: int64

In [48]:
## no duplicates
cons_df.duplicated().value_counts()

False    3014
dtype: int64

In [52]:
## Save consolidated data to a csv
cons_df.to_csv('guitar_data_consolidated.csv', index=False)

Now that the data has been consolidated EDA and feature analysis can take place. Please see Part 2 for that process.